# Szyfrowanie + kompresja
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Konfiguracja tunelu VPN/sprawozdanie.md](https://github.com/Gombek7/techniki-poufnosci/blob/main/Konfiguracja%20tunelu%20VPN/sprawozdanie.md)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - własny algrytm szyfrowania/deszyfrowania

Wybrany algorytm to szyfr Cezara.

### Szyfrator i deszyfrator

Najpierw został zdefiniowany alfabet, czyli zestaw znaków obsługiwanych przez nasz algorytm.

In [9]:
const ALPHABET = "AĄBCĆDEĘFGHIJKLŁMNŃOÓPRSŚTUVWXYZŹŻaąbcćdeęfghijklłmnńoóprsśtuvwxyzźż\b\t\n\f\r !\"#$%&'()*+,-./0123456789:;<=>?@[\\]^`{|}~"

Następnie została napisana funkcja szyfrująca `encrypt`.

In [10]:
function encrypt(input: string, key: number): string
{
    const output: string[] = [];
    const n = ALPHABET.length; //liczba znaków w alfabecie
    for (const char of input) {
        const a = ALPHABET.indexOf(char); //indeks znaku do zaszyfrowania
        const c = (a + key) % n ;         //indeks zaszyfrowanego znaku
        output.push(ALPHABET[c]) ;        //zamiana indeksu na znak i wpisanie do wyjścia
    }
    return output.join("");
}

Funkcja deszyfrująca `decrypt` jest podobna, z tą różnicą, że klucz jest odejmowany od indeksu znaku.

In [63]:
function decrypt(input: string, key: number):string
{
    const output: string[] = []
    const n = ALPHABET.length //liczba znaków w alfabecie
    for(const letter of input)
    {
        const c = ALPHABET.indexOf(letter) //indeks zaszyfrowanego znaku
        const a = (c + n - key) % n        //indeks odszyfrowanego znaku
        output.push(ALPHABET.at(a))        //zamiana indeksu na znak i wpisanie do wyjścia
    }
    return output.join("")
}

Poniżej znajduje się krótki test szyfru. Podany tekst jest zaszyfrowywany losowym kluczem. Następnie odszyfrowywany prawidłowym i nieprawidłowym kluczem.

In [89]:
import { randomIntegerBetween } from 'jsr:@std/random';

const input = "Cześć! To jest tekst oryginalny!";
const key = randomIntegerBetween(0, 2 * ALPHABET.length);
const invalidKey = key + randomIntegerBetween(1, Math.ceil(ALPHABET.length / 2));

const encrypted = encrypt(input, key);
const decryptedInvalid = decrypt(encrypted, invalidKey);
const decryptedValid = decrypt(encrypted, key);

Deno.jupyter.md`| **Tekst oryginalny** | ${input} |
| --- | --- |
| Prawidłowy klucz | ${key} |
| Nieprawidłowy klucz | ${invalidKey} |
| **Tekst zaszyfrowany** | **${JSON.stringify(encrypted)}** |
| Tekst odszyfrowany z nieprawidłowym kluczem | ${decryptedInvalid} |
| **Tekst odszyfrowany z prawidłowym kluczem** | **${decryptedValid}** |
`

| **Tekst oryginalny** | Hello! This is the input! |
| --- | --- |
| Prawidłowy klucz | 223 |
| Nieprawidłowy klucz | 237 |
| **Tekst zaszyfrowany** | **"CŻęęjżźŃcćmźćmźńcŻźćhlońż"** |
| Tekst odszyfrowany z nieprawidłowym kluczem | {UaadutIYZgtZgtiYUtZcęjiu |
| **Tekst odszyfrowany z prawidłowym kluczem** | **Hello! This is the input!** |


## Łamanie metodą brute force

Przypuśćmy, że atakujący wie, że zna alfabet, w którym została napisana wiadomość. Może go uzyskać poprzez przechwycenie kilku zaszyfrowanych wiadomości. Atakujący może z łatwością znaleźć wszystkie możliwe odszyfrowane wiadomości, jednak problemem może być rozpoznanie prawidłowej wiadomości. Jednym ze soposobów jest dopasowanie częstości liter do języka, w którym zostana napisana wiadomość.